# Data validation
This is a mostly manual section, where we check data validity and inconsistencies. Also, it is important for the data to be resilient to bugs.

In [257]:
import os
import pickle as pkl
import re
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# %matplotlib inline
warnings.filterwarnings('ignore')

# define functions

In [258]:
# get numerical columns
def get_numerical_mask(df):
    type_mask = []
    for i in df.dtypes:
        if str(i).startswith('float') or str(i).startswith('int'): # or str(i).startswith('bool')
            type_mask.append(True)
        else: type_mask.append(False)
    num_cols = list(np.array(df.columns)[type_mask])
    other_cols = list(np.array(df.columns)[[not elem for elem in type_mask]])
    
    return num_cols, other_cols

def duplicate_cleanup(df, col_id):
    original_size = df.shape[0]
    col_subset = df.columns.to_list()
    n_duplicates = df[(df.duplicated(col_subset))].shape[0]
    df.drop_duplicates(col_subset, inplace=True)
    print('number of removed duplicates:', n_duplicates)
    
    true_duplicates = df[(df.duplicated(col_id))].shape[0]
    if true_duplicates != 0:
        raise ValueError('There are still duplicates to verify')
        
    return df

# Define paths and capture data

In [259]:
inputs = os.path.join('..', 'data', '01_raw')
outputs = os.path.join('..', 'data', '02_intermediate')
reports = os.path.join('..', 'data', '06_reporting')

data = pd.read_csv(os.path.join(inputs, 'WA_Fn-UseC_-Telco-Customer-Churn.csv'))
# data_identity = pd.read_csv(os.path.join(inputs, 'train_identity.csv'))

In [260]:
print('shape of data:', data.shape[0])
data.head()

shape of data: 7043


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# standardize data

#### columns

In [261]:
data.rename({c: c.lower() for c in data.columns}, axis=1, inplace=True)
data.rename({'churn': 'y', 'customerid': 'id'}, axis=1, inplace=True)

#### strip values of categorical

In [262]:
for cat in data.columns:
    if data[cat].dtypes == 'object':
        data[cat] = data[cat].apply(lambda x: x.strip().lower())

# check duplicates for main data

In [263]:
data = duplicate_cleanup(data, 'id')

number of removed duplicates: 0


# set index

In [264]:
data.set_index('id', inplace=True)

# build new variables

In [265]:
# todo

# drop unused columns

# boolean features to float (0,1)

In [266]:
boolean_vars = ['partner', 'dependents', 'phoneservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 
            'techsupport', 'streamingmovies', 'paperlessbilling', 'y']
for col in boolean_vars:
    data[col] = data[col].map( {'no': 0, 'yes': 1} ).astype(float)
    
data['y'] = data['y'].astype(int)
data['gender'] = data['gender'].map( {'female': 0, 'male': 1} ).astype(float)
data.rename({'gender': 'gender_male'}, axis=1, inplace=True)

# fix non-categorical types

In [267]:
data.loc[data['totalcharges'] == '', ['totalcharges']] = np.nan
data['totalcharges'] = data['totalcharges'].astype('float')

# evaluate categorical features

In [268]:
categories_dict = {}
for cat in data.columns:
    if data[cat].dtypes == 'object':
        categories_dict[cat] = list(data[cat].unique())
        n_categories = data[cat].unique().size
        print("number of categories for variable '{}': {} ".format(cat, n_categories))

number of categories for variable 'multiplelines': 3 
number of categories for variable 'internetservice': 3 
number of categories for variable 'streamingtv': 3 
number of categories for variable 'contract': 3 
number of categories for variable 'paymentmethod': 4 


# report data types

In [269]:
numerical_cols, other_cols = get_numerical_mask(data)
# numerical_cols.remove('y')
print(numerical_cols)

num_df = pd.DataFrame(numerical_cols)
other_df = pd.DataFrame(other_cols)

['gender_male', 'seniorcitizen', 'partner', 'dependents', 'tenure', 'phoneservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingmovies', 'paperlessbilling', 'monthlycharges', 'totalcharges', 'y']


#### concatenate

In [270]:
report_df = pd.concat([num_df,other_df], ignore_index=True, axis=1)
report_df.columns = ['numerical_cols', 'non_numerical_cols']

# Checking  for possible anomalies in the database
Here we check for some possible anomalies. If there was one, we should look for solutions such as: contact the team responsible for data engineering or anyone who has a good understanding of the provided data, search for flaws in the data pipeline, or at least document for those anomalies.

Check if there is any oddity in data.

In [271]:
data.loc[:, ].describe()

,gender_male,seniorcitizen,partner,dependents,tenure,phoneservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingmovies,paperlessbilling,monthlycharges,totalcharges,y
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,5517.000000,5517.000000,5517.000000,5517.000000,5517.000000,7043.000000,7043.000000,7032.000000,7043.000000
mean,0.504756,0.162147,0.483033,0.299588,32.371149,0.903166,0.365960,0.440276,0.439007,0.370491,0.495197,0.592219,64.761692,2283.300441,0.265370
std,0.500013,0.368612,0.499748,0.458110,24.559481,0.295752,0.481742,0.496465,0.496311,0.482980,0.500022,0.491457,30.090047,2266.771362,0.441561
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,18.800000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.500000,401.450000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,70.350000,1397.475000,0.000000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,89.850000,3794.737500,1.000000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,118.750000,8684.800000,1.000000


# save data

In [272]:
data.head()

,gender_male,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,y
id,,,,,,,,,,,,,,,,,,,,
7590-vhveg,0.0,0,1.0,0.0,1,0.0,no phone service,dsl,0.0,1.0,0.0,0.0,no,0.0,month-to-month,1.0,electronic check,29.85,29.85,0
5575-gnvde,1.0,0,0.0,0.0,34,1.0,no,dsl,1.0,0.0,1.0,0.0,no,0.0,one year,0.0,mailed check,56.95,1889.50,0
3668-qpybk,1.0,0,0.0,0.0,2,1.0,no,dsl,1.0,1.0,0.0,0.0,no,0.0,month-to-month,1.0,mailed check,53.85,108.15,1
7795-cfocw,1.0,0,0.0,0.0,45,0.0,no phone service,dsl,1.0,0.0,1.0,1.0,no,0.0,one year,0.0,bank transfer (automatic),42.30,1840.75,0
9237-hqitu,0.0,0,0.0,0.0,2,1.0,no,fiber optic,0.0,0.0,0.0,0.0,no,0.0,month-to-month,1.0,electronic check,70.70,151.65,1


In [273]:
data.to_csv(os.path.join(outputs, 'data.csv'))